# A brief introduction to the scikit-learn machine learning environment

In this module, we will work through the steps of retrieving data on biomedical literature articles in PubMed and we will introduce ourselves to the very comprehensive machine learning environment, ***scikit-learn***.

Specifically we will:
1. Create a simple function that does a lot of work: when passed a proper PubMed query, it will return a dictionary full of PMIDs and their abstracts generated by PubMed. 
2. Learn how to take information stored in dictionaries, like the abstracts above,and write them to a directory as text files on disk. These files can be used by really any program that is expecting a directory of texts (scikit-learn, BRAT, etc.)
3. See how to do some basic NLP text preprocessing like i) filtering out stop words; ii) tokenization; iii) mapping all characters to lowercase; iv) and stemming.
4. Finally, we will run some simple machine learning exercises on the abstracts we extracted in Part 1.


In [ ]:
# skip this cell if you already have executed it in a previous notebook
#import nltk
#nltk.download() #enter 'd' at 'Downloader>' prompt; then enter 'book' at 'Identifier>' prompt; enter 'q' to quiit.

#import nltk
#nltk.download('perluniprops')

In [ ]:
from Bio import Entrez, Medline
import nltk
import gensim
import operator
import pandas as pd
import re

## Using the Biopython package to retreive abstracts from PubMed
<p><font color='darkblue'>Biopython is a Python package designed for computational molecular biology and bioinformatics. It mainly contains tools useful for analyzing genetic and sequence data, but it also contains a set of functions that allow us to query against PubMed easily. </font></p>

### But first: Why would we want to focus on abstracts? 
<p><font color='darkblue'>Abstracts from articles in the biomedical literature are useful for a number of purposes. One of the most important is that reviewing abstracts is usually the first step  in building a systematic review.

**First, we need a function that collects records from PubMed when given a proper PubMed search query:**

In [ ]:
def queryAbstracts(search_terms : str, max = 5):    
    '''This function takes a string containing the usual PubMed terms we would use with
    PubMed itself and returns a dictionary of matching PubMed IDs (dictionary keys) and matching PubMed 
    abstracts (dictionary values) that PubMed returns when queried with search_terms, limited to "max" abstracts.
    
    It handles all the messy URL communication AND the messy XML format returned by PubMed. Credit the
    Biopython package for making that so easy...
    '''
    Entrez.email = "john.hurdle@utah.edu" #NLM likes to know who is sending requests to its PubMed APIs
    Entrez.tool = "Biopython" #recommended but not necessary
    handle = Entrez.esearch(db="pubmed", term = search_terms, retmax = max) #these next 4 lines talk 
    record = Entrez.read(handle)                                            #to PubMed to find IDs matching search terms
    idlist = record["IdList"]
    handle.close()
    
    handle = Entrez.efetch(db="pubmed", id=idlist, rettype="medline", retmode="text") #these next 10 lines talk 
    records = Medline.parse(handle)                                                   #talk to PubMed to query for the abstracts
    abstracts=dict()  # create the empty dictionary
    for record in records: #for each ID returned above, see if it has an abstract (e.g., letters to editor don't)
        if 'AB' in record: #if it does, entry text of abstract under the PMID as key in dictionary
            pmid=record['PMID']
            abstracts[pmid]=record['AB']
        if len(abstracts)== max: #once we hit max, we quit
            break;
    handle.close()
    
    return abstracts

<p><font color='darkblue'>Let's try to search for a set of abstracts that can be compared to each other for classification purposes. First, we will grab up to 200 abstracts in PubMed for which the National Library of Medicine librarians have indexed the articles under the MeSH headings of "natural language processing". **This is an example of using an external data source to add value to an ML task.**
    
Here we are using the skill of the NLM librariians to establish the refrence (or "gold") standard, in lieu of annotation in clinical notes. We focus here on abstracts, but we could, with a little more effort, use full-text articles from PubMedCentral.

In [ ]:
abstracts_NLP = queryAbstracts('"natural language processing"[mesh]"',200) # You can put any query that the work in PubMed inside the single quotes here…
len(abstracts_NLP)

<p><font color='darkblue'>Then we will grab up to 200 abstracts in PubMed for which the National Library of Medicine 
librarians have indexed the articles under the MeSH headings of "artificial intelligence".

In [ ]:
abstracts_AI = queryAbstracts('"artificial intelligence"[MeSH Terms] \
                               NOT "natural language processing"[All Fields]',200)
len(abstracts_AI)

### Why choose "artificial intelligence? Why do we need the NOT in the AI query?
**The MeSH tree for "natural language processing":**

    All MeSH Categories
        Information Science Category
            Information Science
                Computing Methodologies
                    Algorithms
                        Artificial Intelligence
                            Natural Language Processing <--- HERE ********************************

**The MeSH tree for "artificial intelligence":**

    All MeSH Categories
        Information Science Category
            Information Science
                Computing Methodologies
                    Algorithms
                        Artificial Intelligence <--HERE ********************************
                            Computer Heuristics
                            Expert Systems
                            Fuzzy Logic
                            Knowledge Bases
                                Biological Ontologies +
                            Machine Learning
                                Supervised Machine Learning +
                                Unsupervised Machine Learning
                            Natural Language Processing <--HERE ********************************
                            Neural Networks (Computer)
                            Robotics


<p><font color='darkblue'>***If it is important to your particular task,*** the code in this next cell will ensure that both directories (files are written to disk further down in the code) are of equal size, that is, that they are equally balanced.

In [ ]:
num_abs_remove = abs(len(abstracts_NLP) - len(abstracts_AI)) #see how many files need to be trimmed from larger set
if num_abs_remove == 0:
    pass #the classes are balanced, full speed ahead
elif len(abstracts_NLP) > len(abstracts_AI): #then remove the extra positive files
    abs_count = 0
    del_list = []
    for del_key in abstracts_NLP.keys():
        del_list.append(del_key)
        abs_count += 1
        if abs_count >= num_abs_remove: 
            for del_key in del_list:
                abstracts_NLP.pop(del_key, None)
            break
else:  #then (abstracts_AI) > len(abstract_NLP): so remove the extra negative files
    abs_count = 0
    del_list =[]
    for del_key in abstracts_AI.keys():
        del_list.append(del_key)
        abs_count += 1
        if abs_count >= num_absL_remove: 
            for del_key in del_list:
                abstracts_AI.pop(delkey, None)
            break
print("num NLP: ", len(abstracts_NLP), " num AI: ", len(abstracts_AI))
        


## 3. Preprocessing the dataset

#### <p><font color='darkblue'>Some common preprocessing tax for clinical text are::<br></font></p>
* **shuffle the documents:** so that repeated runs use a new ordering (keeps us honest...) **Doh, scikit can do this, too**
* **lowercase words:** so words like "Library" and "library" look the same to the ML ***(any problems with this idea?)*** **Doh, scikit can do this, too**
* **stem the words:** so words like "blood", "bled", bleeding" look the same to the ML
* **remove stop words:** so common words like "a", "the", etc. are not considered by the ML **Doh, scikit can do this, too**
* **remove digits:** so numbers don't inlfuence the ML

It would be enormously useful to have a pre-processors tool that ** expands acronyms and short forms**, But that is harder than it sounds to do really well.


In [ ]:
#This code sets up a string of punctuation characters and a useful list of stopwords.
import string
from nltk.corpus import stopwords
nltk.download('nonbreaking_prefixes')
from nltk.tokenize.moses import MosesTokenizer
from nltk.stem.snowball import SnowballStemmer

punct = set(string.punctuation)
my_stopwords = set(stopwords.words('english'))
print(my_stopwords)

### We won't run this now since scikit can tokenize and stem the input text if we want. But it is better to make pre-processing options obvious when we run the data fetch, but I include the code for your reference.

Tokenize and stem all words in positive abstracts, then put it back into the abstract text field

In [ ]:
# tokenize and stem all words in positive abstracts, then put it back into the absrtact text field
stemmer = SnowballStemmer("english")
tokenizer = MosesTokenizer()

for abstract_id in doc_ids_NLP:  #loop through all of the shuffled positive abstracts
    processed_abstract = '' #start with a blank string for each abstract
    for word in tokenizer.tokenize(abstracts_NLP[abstract_id]): #tokenize and stem each word.
        if not word.replace('-','',1).isdigit() \
        and not word in my_stopwords: \
            processed_abstract += ' ' +stemmer.stem(word.lower()) #concatenate stemmed word into new abstract
    abstracts_NLP[abstract_id] = processed_abstract  #replace non-processed text with processed text
##!! Need to write full (non-stemmed, non lower case)text files automatically under 'abstracts_full_<pos | neg>

### Ditto, not run here...
Tokenize and stem all words in negative abstracts, then put it back into the abstract text field

In [ ]:
# tokenize and stem all words in negative abstracts, then put it back into the abrstact text field
for abstract_id in doc_ids_AI:  #loop through all of the abstracts
    processed_abstract = '' #start with a blank string for each abstract
    for word in tokenizer.tokenize(abstracts_AI[abstract_id]): #tokenize and stem each word.
        if not word.replace('-','',1).isdigit() \
        and not word in my_stopwords: \
            processed_abstract += ' ' +stemmer.stem(word.lower()) #concatenate stemmed word into new abstract
    abstracts_AI[abstract_id] = processed_abstract  #replace non-processed text with processed text
##!! Need to write full (non-stemmed, non lower case) text files automatically under 'abstracts_full_<pos | neg>

### Now write (or re-write) the pre-processed files to directories so we can import them into scikit-learn or into BRAT or wherever we want...
<p><font color='darkblue'>Note that there is a hardcoded flag, called dir_overwrite, that prevents files from being accidentally overwritten. If this entire notebook were wrapped up into a program, then that flag would likely be set by a function call for input statement to the user.

In [ ]:
import os
import shutil as sh

###############################
dir_overwrite = False  # set to True if you REALLY want to overwrite your previous files
###############################

my_dir ="./data/"  
if not os.path.exists(my_dir): #check to see inthe data directory exists; if not, make it
    os.mkdir(my_dir)
path_NLP = my_dir+"abstracts_NLP/"
path_AI = my_dir+"abstracts_AI/"

In [ ]:
if os.path.exists(path_NLP) & dir_overwrite:  # then remove old dir the positive abstracts file
    sh.rmtree(path_NLP)  #remove the old dir and its files
    print("re-writing positive files\n")
if os.path.exists (path_AI) & dir_overwrite:  # then overwirte the positive abstracts file
    sh.rmtree(path_AI)  #remove the old dir and its files
    print("re-writing negative files\n") 
if not os.path.exists(path_NLP):
    os.mkdir(path_NLP)#make new empty dir
if not os.path.exists(path_AI):
    os.mkdir(path_AI)#make new empty dir

In [ ]:
if dir_overwrite:  # only overwite bot sets of files if overwrite is True
    for key in abstracts_NLP.keys(): #loop trhough each positve abstract and build a text file
        text = abstracts_NLP[key]  #get the text for this pubmed id
        myfile = open(path_NLP+key+".txt",'w') #write the text under a file name of the pubmed id
        myfile.write(text)
        myfile.close()
    print ("--> Finished writing positives...\n")
    
    for key in abstracts_AI.keys(): #loop trhough each negative abstract and build a text file
        text = abstracts_AI[key]  #get the text for this pubmed id
        myfile = open(path_AI+key+".txt",'w') #write the text under a file name of the pubmed id
        myfile.write(text)
        myfile.close()
    print ("--> Finished writing negatives...\n")

### ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
### Okay, we are ready to play with sci-kit-learn!
### ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

### First:
<p><font color='darkblue'>We load the files into scikit-learn. The scikit designers did something very clever here: they just require you organize the target texts into separate directories by class type. So here all of the positive abstracts (i.e., NLP articles) are in one directory and all the negative (i.e., AI but not NLP) articles are in another. If you have three classes to classify, you'd have three directories. The name of the sub-directory becomes the class name in the classifer model. Neat.

In [ ]:

import sklearn
from sklearn import datasets
my_dir = "./data/" 
my_bunch = sklearn.datasets.load_files(my_dir, #scikit uses subdir names as class names \
                            description = "This dir holds two sets of abstracts. The abstract_<pos | neg> holds ~200 files each of text of PubMed abstracts that are positive or negative for some category, where each token in files is stemmed and lowercase. The abstract_full_<pos | neg> holds same text files with no stemming or lower case.", \
                            categories = None, #just process the subdirs as classes \
                            load_content = True, #loads actual text into scikit -- not sure what this means if False \
                            shuffle = True,      #mixes up order to keep the ML honest \
                            encoding = 'utf-8',  #required flag for text files but could be none for image files or other non-text) \
                            decode_error = 'strict',#optional \
                            random_state = 0) # can change to another integer to get different start state 


#### --as a sanity check, let's print some stats on the "bunch", the scikit-learn basic data structure.

In [ ]:
#print(my_bunch.target)
print(my_bunch.target_names)
print("sample target values: ",my_bunch.target[0:10])
for t in my_bunch.target[:10]:
    print(my_bunch.target_names[t], " | ", end = "")
print("\n",len(my_bunch.filenames)) #count number of file names
print(len(my_bunch.data), "-- should be same as above")  # count number of files' data sets -- should be same as above
print("\n".join(my_bunch.data[0].split("\n")[:3]))

### Second:
<p><font color='darkblue'>We specifiy a tf-idf vector maker as our first pass at features for our machine learning task. We can do a lot of **pre-processing** here using the options below...

In [ ]:
# You can safely ignore any parameter here that have no comments
# Details are here: http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)
from sklearn.feature_extraction.text import TfidfVectorizer

#First, make a TfidfVectorizer instance
vectorizer = TfidfVectorizer(#input = my_bunch, # From the load files above\
                        encoding = 'utf-8', \
                        decode_error = 'strict', \
                        strip_accents = None, \
                        lowercase = True, \
                        preprocessor = None, #could put our stemming function here \
                        tokenizer = None, #could put our stemming or tokenizer here \
                        analyzer = 'word', # could be 'char' for character models \
                        stop_words = 'english', #use None if no stop words desired \
                        token_pattern = '(?u)\b\w\w+\b', \
                        ngram_range = (1, 2), #use 1's here for unigrams, 2's for bigrams... \
                        max_df = 0.1, min_df = 1, \
                        max_features = None, #can limit the length of feature vectors here, otherwise use them all\
                        vocabulary = None, \
                        binary = False, \
                        dtype = 'numpy.int64', \
                        norm = 'l2', #L2 is standard "norming function" to use here\
                        use_idf = True, \
                        smooth_idf =  True, \
                        sublinear_tf= False)


### Third:
<p><font color='darkblue'>We use scikit to count each token across the corpus of positive and negative abstracts. **Bag of Words!**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
#make a vectorizer instance
count_vect = CountVectorizer() 
#apply it to our data
my_train_counts = count_vect.fit_transform (my_bunch.data)
my_train_counts.shape

### Fourth:
<p><font color='darkblue'>We use the scikit tf-idf vectorizer that we defined above to convert the counts into tf-idf values.

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
#make a vectorizer instance 
tfidf_transformer = TfidfTransformer()
#apply it to our data
my_train_tfidf = tfidf_transformer.fit_transform(my_train_counts)
my_train_tfidf.shape

### Now it is time to choose a machine learning model. 
<p><font color='darkblue'>So let's start with the universally recommended Naive-Bayes model for the baseline. Scikit makes changing models really simple: just change these two lines.

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(my_train_tfidf,my_bunch.target)

<p><font color='darkblue'>At this point we could build a proper test corpus, but it would be hard to see in a cell why a given abstract would be classified one way or the other. So we'll just make up a few toy abstracts.

In [ ]:
test_abstracts = ['This article uses natural language processing to push science forward', \
   
                  'Robots push society backwards', \
                  
                  "Roses are red, violets are snappy, if NLP was easy I'd be happy!",\
                 
                  'Micro blots are fun', \
                 
                  '[Actual AI/non-NLP abstract not seen yet]Details are given of the application of Artificial Neural Networks (ANNs) to predicting the compliance of bathing waters along the coastline of the Firth of Clyde, situated in the south west of Scotland, UK. Water quality data collected at 7 locations during 1990-2000 were used to set up the neural networks. In this study faecal coliforms were used as a water quality indicator, i.e. output, and rainfall, river discharge, sunlight and tidal condition were used as input of these networks. In general, river discharge and tidal ranges were found to be the most important parameters that affect the coliform concentration levels. For compliance points close to the meteorological station, the influence of rainfall was found to be relatively significant to the concentration levels.' \
                 ]
X_new_counts = count_vect.transform(test_abstracts)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(test_abstracts, predicted):
    print('\n%r => %s' % (doc, my_bunch.target_names[category]))

### Now, for somethiing toally cool: let's run  some scikit code that will take the abstract files from above, read them in, and run a pipeline that ***repetitively*** executes the Naive Bayes model while trying different parameters for that model, to see which parameter combination works the best for this data set. This is called "parameter search". You can substitute other scikit learning models in line "`('clf', MultinomialNB())`" BUT you uneed to find the parameters for the model in the scikit documentation and specificy them in the "`parameters = {...`" dictionary code below. <p><font color='darkblue'>NB: this takes about a minute to run and generates a lot of warnings...scroll down to the resuts at the bottom.
    

In [ ]:
from __future__ import print_function

from pprint import pprint
from time import time
import logging

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import sklearn
from sklearn import datasets

#print(__doc__)

# Display progress logs on stdout
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(message)s')


# #############################################################################
# Load some categories from the training set
categories = None

# Uncomment the following to do the analysis on all the categories
#categories = None

print("Loading abstracts...") #print("Loading 20 newsgroups dataset for categories:")
#print(categories)
my_dir = "./data/" 
data = sklearn.datasets.load_files(my_dir, #scikit uses subdir names as class names \
                                   description = "This dir holds two sets of abstracts. The abstract_<pos | neg> holds ~200 files each of text of PubMed abstracts that are positive or negative for some category, where each token in files is stemmed and lowercase. The abstract_full_<pos | neg> holds same text files with no stemming or lower case.", \
                                   categories = None, #just process the subdirs as classes \
                                   load_content = True, #loads actual text into scikit -- not sure what this means if False \
                                   shuffle = True,      #mixes up order to keep the ML honest \
                                   encoding = 'utf-8',  #required flag for text files but could be none for image files or other non-text) \
                                   decode_error = 'strict',#optional \
                                   random_state = 0)
#data = fetch_20newsgroups(subset='train', categories=categories)
print("%d documents" % len(data.filenames))
print("%d categories" % len(data.target_names))
print()

# #############################################################################
# Define a pipeline combining a text feature extractor with a simple
# classifier
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])

# uncommenting more parameters will give better exploring power but will
# increase processing time in a combinatorial way
parameters = {
    'vect__max_df': (0.05, 0.1, 0.3, 0.4,0.5, 0.75, 1.0),
    #'vect__max_features': (None, 5000, 10000, 50000),
    'vect__ngram_range': ((1, 1), (1, 2), (1,3),(1,4), (1,5), (1,6)),  # unigrams or bigrams
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    #use for SGDClassifier()...'clf__alpha': (0.00001, 0.000001),
    #'clf__penalty': ('l2', 'elasticnet'),
    #'clf__n_iter': (10, 50, 80),
}

if __name__ == "__main__":
    # multiprocessing requires the fork to happen in a __main__ protected
    # block

    # find the best parameters for both the feature extraction and the
    # classifier
    grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)

    print("Performing grid search...")
    print("pipeline:", [name for name, _ in pipeline.steps])
    print("parameters:")
    pprint(parameters)
    t0 = time()
    grid_search.fit(data.data, data.target)
    print("done in %0.3fs" % (time() - t0))
    print()

    print("Best score: %0.3f" % grid_search.best_score_)
    print("Best parameters set:")
    best_parameters = grid_search.best_estimator_.get_params()
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))